In [1]:
import findspark
findspark.init()

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Spark session & context
spark = SparkSession.builder.master("spark://spark:7077") \
        .appName("jupyter-notebook") \
        .config("spark.driver.memory", "512m") \
        .config("spark.mongodb.write.connection.uri", "mongodb://mongodb:27017/test.myCollection") \
        .config("spark.mongodb.read.connection.uri", "mongodb://mongodb:27017/test.myCollection") \
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \
        .getOrCreate()
spark

In [2]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark:7077 appName=jupyter-notebook>

In [3]:
# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(1000+1))
rdd.sum()

500500

# WOW
Nose


In [4]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", 50)], schema='name string, age int')

people.write.format("mongodb").mode("append").save()

# If you need to write to a different MongoDB collection, use the .option() 
# method with .write().
# To write to a collection called contacts in a database called people, 
# specify the collection and database with .option():
# people.write.format("mongodb").mode("append").option("database","people").option("collection", "contacts").save()

people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|  50|
+-------------+----+



In [5]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|  50|
+-------------+----+



In [6]:
people.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [7]:
df = spark.read.format("mongodb").load()
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [47]:
df.show()
# dt = spark.read.format("mongodb").load()
# x = df.filter(df['age'] >= 10)
# print(x.show(vertical=False))



Py4JJavaError: An error occurred while calling o294.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 4 times, most recent failure: Lost task 0.3 in stage 25.0 (TID 64) (192.168.32.2 executor 0): java.io.InvalidClassException: org.apache.spark.util.collection.BitSet; local class incompatible: stream classdesc serialVersionUID = 250628929810266153, local class serialVersionUID = 59660257146063567
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:699)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2005)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1852)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2005)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1852)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2186)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1669)
	at java.io.ObjectInputS

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 33344)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/conda/lib/python3.8/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/opt/conda/lib/python3.8/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/opt/conda/lib/python3.8/site-packages/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(sel

In [ ]:
spark.stop()